# Spark Optimisation - Shuffle Partitions

## What's Shuffling?
- It happens when there's a wide transformation
- It happens when Spark needs to get the data that resides in different nodes in one place
- Say for instance, you want to find the average transaction amount across all transactions then Spark will need to gather all it's partitions that are split across different nodes and then aggregate - this movement of data is called shuffle

## What is Shuffle partitions
- It simply refers to the partitions made during the shuffle operation
- It is set using

```python
spark.sql.shuffle.partitions (default=200)
spark.default.parallelism

## So Why is it important to decide the No. of Shuffle Partitions?
**Two Main Reasons**
- In order to ensure optimal utlisation of resources
- To improve job performance

### Scenario 1
- Number of executors = 5
- Number of Cores in each Executor = 4
- Shuffle Data Size = 300 GB

- Total no. of cores = 5*4 = 20
- Default no. of shuffle partition = 200
- Size per shuffle partition = 300 GB / 200 = 1.5 GB
  
- ***Note that optimal partition size should be between 1 to 200 MB*** => So we need to tune the number of shuffle partitions
    - If we want the size of partition to be 200 MB then we would need: (300 * 1000)/200 = **1500 Shuffle partitions**

### Scenario 2
- Number of executors = 3
- Number of Cores in each Executor = 4
- Shuffle Data Size = 50 MB
- Total no. of cores = 3*4 = 12
- Default no. of shuffle partition = 200
- Size per shuffle partition = 50 MB / 200 = 250 KB
- ***Now this size is too small***
    - You can choose the size of partition and based on that get the no. of shuffle partitions
    - **OR**
    - In order to use all cores, 50 MB / 12 = 4.2 MB => with this shuffle partition size you can utlise all the cores



### 1. **Understand the Factors Affecting Shuffle Partitions**
   - **Data Size**:  
     Larger datasets typically require more shuffle partitions to distribute the workload evenly and avoid memory bottlenecks.
   - **Cluster Resources**:
     - Number of cores: More cores allow for higher parallelism.
     - Available memory: Ensure each partition is small enough to fit comfortably in memory to avoid disk spill.
   - **Task Granularity**:
     - Too few partitions: Results in under-utilization of cluster resources and large tasks that can cause out-of-memory errors.
     - Too many partitions: Results in many small tasks, increasing scheduling overhead and shuffle I/O costs.

---

### 2. **Start with Default Values**
   - For **DataFrame and SQL API**:  
     The default value for `spark.sql.shuffle.partitions` is **200**.
   - For **RDD API**:  
     The default parallelism is typically **2 × the total number of cores** in the cluster.

---

### 3. **Refine Based on Data Size**
A general heuristic:
   - **Partition Size**: Aim for **100 MB to 200 MB** per partition.  
     This ensures partitions are large enough to minimize overhead but small enough to fit in memory.
   - Formula:  
     \[
     \text{Shuffle Partitions} = \frac{\text{Total Data Size (in bytes)}}{\text{Desired Partition Size (in bytes)}}
     \]
     Example: If your data is 1 TB and you want ~128 MB partitions:
     \[
     \text{Shuffle Partitions} = \frac{1 \, \text{TB}}{128 \, \text{MB}} = 8192 \, \text{partitions.}
     \]

---

### 4. **Analyze Task Performance**
Use the **Spark UI** to monitor:
   - **Task Duration**: 
     - Long-running tasks suggest partitions are too large.
   - **Task Failure (Out-of-Memory Errors)**: 
     - Indicates partitions are too large to fit in memory.
   - **Shuffle Write/Read Metrics**:
     - Excessive shuffle write/read indicates unnecessary overhead, potentially from too many partitions.

---

### 5. **Tune Dynamically**
Adjust shuffle partitions dynamically based on runtime behavior:
   - **DataFrame and SQL API**:
     ```python
     spark.conf.set("spark.sql.shuffle.partitions", <new_value>)
     ```
   - **RDD API**:
     Use `repartition()` or `coalesce()`:
     - `repartition(n)`: Increases the number of partitions.
     - `coalesce(n)`: Reduces the number of partitions without a shuffle.

---

### 6. **Cluster Resource Awareness**
   - Number of partitions should generally be **2-3 times the total number of cores** in the cluster.  
     Example: For a cluster with 16 cores, aim for **32-48 partitions**.

---

### 7. **Workload-Specific Tuning**
   - **Batch Processing**:  
     Higher shuffle partitions to maximize parallelism (e.g., >200 for large datasets).
   - **Interactive Queries**:  
     Lower shuffle partitions to reduce latency.
   - **Joins or Aggregations**:  
     Fine-tune based on data skew and size. Use **adaptive query execution (AQE)** to dynamically adjust partitions.

---

### 8. **Enable Adaptive Query Execution (AQE)**
If using Spark 3.0 or later, enable **AQE** to let Spark dynamically optimize shuffle partitions based on runtime statistics:
   ```python
   spark.conf.set("spark.sql.adaptive.enabled", "true")
   ```
   Key AQE features:
   - Dynamically adjusts the number of shuffle partitions.
   - Handles data skew automatically.

---

### Example: Tuning Shuffle Partitions
1. Data Size: 1 TB
2. Partition Size: ~128 MB
3. Formula: \( 1 \, \text{TB} / 128 \, \text{MB} = 8192 \)
4. Cluster Resources:  
   -ssary.

---

Would you like help estimating the partitions for a specific Spark job or dataset?